In [4]:
import os, re, json, glob


In [5]:
FOLDER_PATH = r"C:\Users\User\Desktop\KULIAH\SEMESTER 5\NLP\FINAL PROJEK\Hasil_TXT\CLEAN_TXT_UTUH\*.txt"
OUTPUT_PATH = "regex_output.json"


In [7]:
import os, re, json, glob

# =============================
# CONFIG
# =============================
FOLDER_PATH = r"C:\Users\User\Desktop\KULIAH\SEMESTER 5\NLP\FINAL PROJEK\Hasil_TXT\CLEAN_TXT_UTUH\*.txt"
OUTPUT_PATH = "hasil_regex_rule_based.json"

# =============================
# DICTIONARIES
# =============================
DICT_AGAMA = ["Islam", "Kristen", "Katolik", "Hindu", "Buddha", "Konghucu"]

DICT_STATUS = [
    "menolak permohonan",
    "mengabulkan permohonan",
    "menerima permohonan",
    "tidak dapat diterima",
    "terbukti secara sah"
]

# =============================
# REGEX
# =============================
REGEX_NOMOR = re.compile(
    r"(?:Nomor|No\.?)\s*[:\-]?\s*([0-9]+\s*(?:PK|K)?\/Pid(?:\.B|\.Sus)?\/[0-9]{4})",
    re.IGNORECASE
)

REGEX_PN = re.compile(
    r"Pengadilan\s+Negeri\s+([A-Za-zÀ-ÖØ-öø-ÿ' .-]{2,50})",
    re.IGNORECASE
)

REGEX_TANGGAL_PUTUSAN = re.compile(
    r"([0-9]{1,2}\s+"
    r"(Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)"
    r"\s+[0-9]{4})",
    re.IGNORECASE
)

REGEX_HAKIM = re.compile(
    r"([A-Z][A-Za-z\s\.]+,\s*S\.H(?:\.,\s*M\.H)?)",
    re.IGNORECASE
)

REGEX_TERDAKWA = re.compile(
    r"Nama\s*:\s*([^\n;]{3,100})",
    re.IGNORECASE
)

REGEX_TGL_LAHIR = re.compile(
    r"(?:Tanggal\s+Lahir|Umur\/Tanggal\s+Lahir)\s*[:\-]?\s*"
    r"[0-9]+\s*Tahun\s*\/\s*"
    r"([0-9]{1,2}\s+"
    r"(Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)"
    r"\s+[0-9]{4})",
    re.IGNORECASE
)

REGEX_PASAL = re.compile(
    r"(Pasal\s+\d+(?:\s+Ayat\s*\(\d+\))?"
    r"(?:\s+ke\-?\d+)?"
    r"(?:\s+juncto\s+Pasal\s+\d+(?:\s+Ayat\s*\(\d+\))?)?\s+KUHP)",
    re.IGNORECASE
)

REGEX_KORBAN = re.compile(
    r"Korban\s+([A-Z][A-Za-z\s]{2,50})",
    re.IGNORECASE
)

# =============================
# HELPER
# =============================
def clean(x):
    return re.sub(r"\s+", " ", x).strip() if x else "-"

def extract_single(regex, text):
    m = regex.search(text)
    return clean(m.group(1)) if m else "-"

def extract_agama(text):
    for a in DICT_AGAMA:
        if a.lower() in text.lower():
            return a
    return "-"

def extract_status(text):
    tail = text[-4000:].lower()
    for s in DICT_STATUS:
        if s in tail:
            return s.title()
    return "-"

def extract_modus(text):
    for s in re.split(r"[.\n]", text):
        if any(k in s.lower() for k in [
            "memukul","menusuk","menikam","menganiaya",
            "menendang","melempar","mencekik"
        ]):
            return clean(s)
    return "-"

def extract_akibat(text):
    t = text.lower()
    if "meninggal dunia" in t or "mati" in t:
        return "Korban meninggal dunia"
    if "luka berat" in t:
        return "Korban mengalami luka berat"
    if "luka" in t:
        return "Korban mengalami luka"
    return "-"

# =============================
# MAIN
# =============================
results = {}

files = glob.glob(FOLDER_PATH)
print("Jumlah file:", len(files))

for fpath in files:
    fname = os.path.basename(fpath)

    with open(fpath, encoding="utf-8", errors="ignore") as f:
        text = f.read()

    results[fname] = {
        "NOMOR_PUTUSAN": extract_single(REGEX_NOMOR, text),
        "LOKASI_PN": extract_single(REGEX_PN, text),
        "NAMA_HAKIM": extract_single(REGEX_HAKIM, text),
        "TANGGAL_PUTUSAN": extract_single(REGEX_TANGGAL_PUTUSAN, text),
        "NAMA_TERDAKWA": extract_single(REGEX_TERDAKWA, text),
        "TANGGAL_LAHIR": extract_single(REGEX_TGL_LAHIR, text),
        "AGAMA": extract_agama(text),
        "PASAL_DAKWAAN": extract_single(REGEX_PASAL, text),
        "VONIS_PENJARA": "-",
        "MODUS": extract_modus(text),
        "KORBAN": extract_single(REGEX_KORBAN, text),
        "AKIBAT": extract_akibat(text),
        "STATUS_PUTUSAN": extract_status(text),
    }

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ Selesai. Output:", OUTPUT_PATH)


Jumlah file: 205
✅ Selesai. Output: hasil_regex_rule_based.json


In [10]:
import json
import pandas as pd

with open("hasil_regex_rule_based.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for fname, fields in data.items():
    row = {"file_id": fname}
    row.update(fields)
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv("pred_regex.csv", index=False)
print("✅ pred_regex.csv dibuat")

✅ pred_regex.csv dibuat


## 🔁 BLEU / ROUGE

In [15]:
import json
import re
from collections import defaultdict

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

nltk.download("punkt")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
def normalize(x):
    if x is None:
        return ""
    if not isinstance(x, str):
        x = str(x)
    x = x.lower()
    x = re.sub(r"\s+", " ", x)
    return x.strip()


In [17]:
with open("dataset.json", "r", encoding="utf-8") as f:
    gt_json = json.load(f)

with open("hasil_regex_rule_based.json", "r", encoding="utf-8") as f:
    pred_json = json.load(f)


In [18]:
gt = defaultdict(lambda: defaultdict(list))

for item in gt_json:
    file_id = str(item["id"])
    for ann in item.get("annotations", []):
        for r in ann.get("result", []):
            text = normalize(r["value"].get("text"))
            for label in r["value"].get("labels", []):
                if text:
                    gt[file_id][label].append(text)


In [20]:
from collections import defaultdict
import re

pred = defaultdict(lambda: defaultdict(list))

def filename_to_id(name):
    nums = re.findall(r"\d+", name)
    return str(205 + int(nums[-1])) if nums else None

for filename, fields in pred_json.items():
    fid = filename_to_id(filename)
    if not fid:
        continue

    for k, v in fields.items():
        if isinstance(v, list):
            for x in v:
                x = normalize(x)
                if x:
                    pred[fid][k].append(x)
        else:
            v = normalize(v)
            if v:
                pred[fid][k].append(v)


In [21]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
smooth = SmoothingFunction().method1

FIELDS = sorted(set(
    f for fid in gt for f in gt[fid]
))

results = []

for field in FIELDS:
    bleu, r1, rL, n = [], [], [], 0

    for fid in gt:
        gt_texts = gt[fid].get(field, [])
        pred_texts = pred.get(fid, {}).get(field, [])

        if not gt_texts or not pred_texts:
            continue

        for g in gt_texts:
            for p in pred_texts:
                g_tok = nltk.word_tokenize(g)
                p_tok = nltk.word_tokenize(p)

                if not g_tok or not p_tok:
                    continue

                bleu.append(sentence_bleu([g_tok], p_tok, smoothing_function=smooth))
                rouge = scorer.score(g, p)
                r1.append(rouge["rouge1"].fmeasure)
                rL.append(rouge["rougeL"].fmeasure)
                n += 1

    results.append({
        "FIELD": field,
        "BLEU": round(sum(bleu)/len(bleu), 4) if bleu else 0,
        "ROUGE-1": round(sum(r1)/len(r1), 4) if r1 else 0,
        "ROUGE-L": round(sum(rL)/len(rL), 4) if rL else 0,
        "N": n
    })


In [22]:
import pandas as pd
df = pd.DataFrame(results).set_index("FIELD")
df


,BLEU,ROUGE-1,ROUGE-L,N
FIELD,,,,
AGAMA,0.0614,0.3476,0.3476,585
AKIBAT,0.0793,0.2278,0.2278,215
KORBAN,0.0103,0.0304,0.0304,207
LOKASI_PN,0.0064,0.0388,0.0388,205
MODUS,0.0011,0.0195,0.0173,265
NAMA_FILE,0.0000,0.0000,0.0000,0
NAMA_HAKIM,0.1877,0.4531,0.4429,1737
NAMA_TERDAKWA,0.0207,0.0637,0.0637,585
NOMOR_PUTUSAN,0.0295,0.4867,0.4867,184
